<a href="https://colab.research.google.com/github/HlibBorovko/KSVD/blob/main/%D0%91%D0%BE%D1%80%D0%BE%D0%B2%D0%BA%D0%BE_%D0%93%D0%BB%D1%96%D0%B1_%D0%9A%D1%81%D0%B2%D0%B4_%D0%BB%D0%B1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision scikit-learn pandas matplotlib seaborn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Завантаження даних
from google.colab import files
uploaded = files.upload()


Saving data.csv to data.csv


In [4]:
df = pd.read_csv("data.csv")
print("Перші рядки датафрейму:")
print(df.head())

Перші рядки датафрейму:
   Phoenix Feather  Unicorn Horn  Dragon's Blood  Mermaid Tears  Fairy Dust  \
0              2.4          18.7            18.4           27.9         7.9   
1              2.1           6.0            15.0           13.3        15.6   
2             17.2          13.9            23.8            6.8        10.7   
3              8.4           9.7             6.8           26.9         4.6   
4             22.1          10.8            16.4           10.5        22.0   

   Goblin Toes  Witch's Brew  Griffin Claw  Troll Hair  Kraken Ink  \
0          9.6          18.3          13.2         2.5        26.0   
1         13.1          11.0           5.0         7.2        26.0   
2         15.8          19.4           2.7        15.4        21.2   
3         29.1          14.6          19.7        18.0        20.8   
4         23.4           2.6          18.2        23.8        11.3   

   Minotaur Horn  Basilisk Scale  Chimera Fang  Cured  
0           10.5        

In [5]:
# Аналіз і підготовка
print("\nІнформація про дані:")
print(df.info())

print("\nПеревірка на пропущені значення:")
print(df.isnull().sum())


Інформація про дані:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2338 entries, 0 to 2337
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Phoenix Feather  2338 non-null   float64
 1   Unicorn Horn     2338 non-null   float64
 2   Dragon's Blood   2338 non-null   float64
 3   Mermaid Tears    2338 non-null   float64
 4   Fairy Dust       2338 non-null   float64
 5   Goblin Toes      2338 non-null   float64
 6   Witch's Brew     2338 non-null   float64
 7   Griffin Claw     2338 non-null   float64
 8   Troll Hair       2338 non-null   float64
 9   Kraken Ink       2338 non-null   float64
 10  Minotaur Horn    2338 non-null   float64
 11  Basilisk Scale   2338 non-null   float64
 12  Chimera Fang     2338 non-null   float64
 13  Cured            2338 non-null   int64  
dtypes: float64(13), int64(1)
memory usage: 255.8 KB
None

Перевірка на пропущені значення:
Phoenix Feather    0
Unicorn Horn       0
D

In [6]:

X = df.drop('Cured', axis=1).values
y = df['Cured'].values

In [7]:
# Масштабування
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
# Поділ на навчальну і тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [9]:
# Перетворення у тензори
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


In [10]:
# Побудова моделі
class LogisticRegressionNN(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionNN, self).__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return torch.sigmoid(self.linear(x))

model = LogisticRegressionNN(X_train.shape[1])
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [11]:
# Навчання моделі
epochs = 100
for epoch in range(epochs):
    for batch_X, batch_y in train_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [10/100], Loss: 0.5052
Epoch [20/100], Loss: 0.1614
Epoch [30/100], Loss: 0.2884
Epoch [40/100], Loss: 0.2502
Epoch [50/100], Loss: 0.4563
Epoch [60/100], Loss: 0.3246
Epoch [70/100], Loss: 0.6885
Epoch [80/100], Loss: 0.4086
Epoch [90/100], Loss: 0.2811
Epoch [100/100], Loss: 0.2973


In [12]:
# Тестування PyTorch-моделі
with torch.no_grad():
    y_pred_nn = model(X_test_tensor)
    y_pred_nn_cls = (y_pred_nn >= 0.5).float()

acc_nn = accuracy_score(y_test, y_pred_nn_cls)
print(f"\nТочність PyTorch моделі: {acc_nn:.4f}")
print("Матриця плутанини:")
print(confusion_matrix(y_test, y_pred_nn_cls))


Точність PyTorch моделі: 0.8910
Матриця плутанини:
[[207  24]
 [ 27 210]]


In [13]:
# Логістична регресія з sklearn
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
y_pred_sk = log_reg.predict(X_test)

acc_sk = accuracy_score(y_test, y_pred_sk)
print(f"\nТочність LogisticRegression з sklearn: {acc_sk:.4f}")
print("Матриця плутанини:")
print(confusion_matrix(y_test, y_pred_sk))


Точність LogisticRegression з sklearn: 0.8932
Матриця плутанини:
[[207  24]
 [ 26 211]]


In [14]:
# Порівняння результатів
print("\nЗвіт PyTorch:")
print(classification_report(y_test, y_pred_nn_cls))

print("\nЗвіт sklearn:")
print(classification_report(y_test, y_pred_sk))


Звіт PyTorch:
              precision    recall  f1-score   support

           0       0.88      0.90      0.89       231
           1       0.90      0.89      0.89       237

    accuracy                           0.89       468
   macro avg       0.89      0.89      0.89       468
weighted avg       0.89      0.89      0.89       468


Звіт sklearn:
              precision    recall  f1-score   support

           0       0.89      0.90      0.89       231
           1       0.90      0.89      0.89       237

    accuracy                           0.89       468
   macro avg       0.89      0.89      0.89       468
weighted avg       0.89      0.89      0.89       468



**Висновок**

У ході лабораторної роботи було реалізовано логістичну регресію двома способамиза допомогою нейронної мережі на PyTorch та класичної моделі зі sklearn. Обидві моделі показали схожі результати, однак нейронна мережа дає більше можливостей для подальшого розширення. Робота дозволила мені закріпити навички з побудови моделей класифікації та роботи з бібліотеками машинного навчання.


